*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
'''
pip install tensorflow-datasets
try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
except Exception:
  pass
'''
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  1
2.8.0-dev20211104


In [2]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

MAXLEN = 100
BATCH_SIZE = 64

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# process the data
columns = ['label', 'text']
train_data = pd.read_table(train_file_path, sep=("\t"), header=None, names=columns)
test_data = pd.read_table(test_file_path, sep=("\t"), header=None, names=columns)

In [4]:
train_data['label'] = train_data['label'].eq('spam').astype(int)
test_data['label'] = test_data['label'].eq('spam').astype(int)
train_labels = train_data.pop('label')
test_labels = test_data.pop('label')

In [5]:
sentences_train = train_data['text'].values
sentences_test = train_data['text'].values

In [6]:
# word embedding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

In [7]:
# vectorize the text
x_train = tokenizer.texts_to_sequences(sentences_train)
x_test = tokenizer.texts_to_sequences(sentences_test)

# get vocab size
vocab_size = len(tokenizer.word_index) + 1

# pad the vecotrized text to equal lengths
x_train = pad_sequences(x_train, padding='post', maxlen=MAXLEN)
x_test = pad_sequences(x_test, padding='post', maxlen=MAXLEN)

In [8]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [9]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
history = model.fit(x_train, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 4s 16ms/step - loss: 0.4097 - acc: 0.8672 - val_loss: 0.4077 - val_acc: 0.8612
Epoch 2/10
105/105 [==============================] - 1s 11ms/step - loss: 0.3943 - acc: 0.8672 - val_loss: 0.4033 - val_acc: 0.8612
Epoch 3/10
105/105 [==============================] - 1s 11ms/step - loss: 0.3928 - acc: 0.8672 - val_loss: 0.3922 - val_acc: 0.8612
Epoch 4/10
105/105 [==============================] - 1s 11ms/step - loss: 0.1257 - acc: 0.9695 - val_loss: 0.1578 - val_acc: 0.9462
Epoch 5/10
105/105 [==============================] - 1s 11ms/step - loss: 0.1097 - acc: 0.9695 - val_loss: 0.1264 - val_acc: 0.9629
Epoch 6/10
105/105 [==============================] - 1s 11ms/step - loss: 0.1021 - acc: 0.9761 - val_loss: 0.1328 - val_acc: 0.9677
Epoch 7/10
105/105 [==============================] - 1s 11ms/step - loss: 0.1076 - acc: 0.9704 - val_loss: 0.1484 - val_acc: 0.9593
Epoch 8/10
105/105 [==============================] - 1s 11ms/step - 

In [10]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = []
  txt = tokenizer.texts_to_sequences([pred_text])
  probabilities = model.predict(txt)
  probability = (probabilities[0][0])
  prediction.append(probability)

  if (probability) >= 0.5:
    prediction.append('spam')
  else:
    prediction.append('ham')
  return (prediction)

In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
